In [4]:
import jax
from jax import numpy as jnp

In [5]:
H = 6
action_dim = 2
N = 1000
key = jax.random.PRNGKey(0)

In [6]:
a_mean = jnp.linspace(0, 1, action_dim*H).reshape((H, action_dim))
a_cov = jnp.tile(jnp.eye(action_dim)*0.04, (H, 1, 1))

sample_key, key = jax.random.split(key)
keys = jax.random.split(sample_key, N)
def single_sample(key, traj_mean, traj_cov):
    return jax.vmap(lambda mean, cov: jax.random.multivariate_normal(key, mean, cov))(traj_mean, traj_cov)
# repeat single_sample N times to get N samples
a_sampled = jax.vmap(single_sample, in_axes=(0, None, None))(keys, a_mean, a_cov)
a_sampled_flattened = jnp.reshape(a_sampled, (N, action_dim*H))

In [7]:
def single_sample(key):
    return jax.random.multivariate_normal(key, a_mean.flatten(), jnp.eye(2*H)*0.04)
sample_key, key = jax.random.split(key)
keys = jax.random.split(sample_key, N)
a_sampled_flattened_new = jax.vmap(single_sample)(keys)

In [ ]:
# get covariance matrix
a_old_cov = jnp.cov(a_sampled_flattened.T)
a_old_cov.shape

In [ ]:
# import pickle

# with open('/home/pcy/Research/quadjax/results/a_sampled_old.pkl', 'rb') as f:
#     a_sampled_old = pickle.load(f)
# with open('/home/pcy/Research/quadjax/results/a_sampled_new.pkl', 'rb') as f:
#     a_sampled_new = pickle.load(f)
# N = 128
# H = 32
# action_dim = 2
# a_sampled_flattened = a_sampled_old.reshape(128, -1)
# a_sampled_flattened_new = a_sampled_new.reshape(128, -1)

In [ ]:
# plot a_sampled_flattened distribution with bars
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
for i in range(action_dim*H):
    plt.figure()
    plt.hist(a_sampled_flattened[:, i], bins=50, alpha=0.5, label='old')
    plt.hist(a_sampled_flattened_new[:, i], bins=50, alpha=0.5, label='new')
    plt.legend()

In [26]:
import jax
from jax import numpy as jnp

N = 1
H = 3
action_dim = 2
n = 5
a_sampled_waypoint = jnp.array(
    [
        [[0.0, 1.0], [2.0, 5.0], [3.0, -1.0]],
    ]
)

# get the linear interpolation of the waypoints (apply to each dimension)
def linear_interpolation(a, b):
    return jnp.arange(n) * (b - a) / n + a
a_sampled = jax.vmap(jax.vmap(jax.vmap(linear_interpolation)))(a_sampled_waypoint[:, :-1], a_sampled_waypoint[:, 1:]) # (N, H-1, action_dim, n)
a_sampled_transposed = jnp.transpose(a_sampled, (0, 1, 3, 2)) # (N, H-1, n, action_dim)
a_sampled_flattened = jnp.reshape(a_sampled_transposed, (N, (H-1)*n, action_dim)) # (N, (H-1)*n, action_dim)

In [27]:
a_sampled_flattened

Array([[[0.        , 1.        ],
        [0.4       , 1.8       ],
        [0.8       , 2.6       ],
        [1.2       , 3.4       ],
        [1.6       , 4.2       ],
        [2.        , 5.        ],
        [2.2       , 3.8       ],
        [2.4       , 2.6       ],
        [2.6       , 1.3999999 ],
        [2.8       , 0.19999981]]], dtype=float32)

In [22]:
a_sampled.reshape(N, (H-1) * action_dim * n)

Array([[0. , 0.4, 0.8, 1.2, 1.6, 2. , 2.2, 2.4, 2.6, 2.8]], dtype=float32)

In [18]:
a_sampled_flattened.shape

(2, 3, 10)